# Pipeline para la geocodificacion de usuarios

La actual pipeline esta pensada para aproximar la geolocalizacion de los usuarios del plan de salud el HI en base a 3 varaibles proveidas: <br>
`provincia_part`, `cp_part`, `loc_part`.<br>
Un analisis exploratorio revelo que usualmente los usuarios escriben campos erroneos en la localidad (a veces el municipio, a veces sencillamente comente errores tipeando) y tambien en la provincia (ej Bariloche anotado como Capital Federal, etc). Encontramos que usualmente los CP son mas fidedignos de la verdadera ubicacion del usuario. Es decir, pueden ser muy precisos, indicando una localidad especifica. Sin embargo a veces no lo son tanto, pudiendo inclusive corresponder a dos provincias al mismo tiempo (mas comun en el interior del pais).

## Preprocessing 

In [2]:
import pandas as pd
import sqlalchemy
import get_centroides as geo # modulo creada para este proyecto. Chequear get_centroides.py

In [3]:
# Base de datos 
df = pd.read_csv('provincias_cp_localidades.csv', encoding='latin-1')
df.head()

,nacionalidad,provincia_part,cp_part,loc_part
0,AR,NaN,1704,NaN
1,AR,NaN,1704,NaN
2,30/06/2015,NaN,CF,1006
3,AR,NaN,1688,12 DE AGOSTO
4,05/08/1987,NaN,BA,1684


In [4]:
df.nacionalidad.str.contains('/').sum()

2

In [5]:
df.nacionalidad.isna().sum()

10

In [6]:
# Vamos a asumir que los campos con valor None pasaran a ser AR. Los campos que tengan fechas en la nacionalidad seran removidos.
df.nacionalidad[df.nacionalidad.isna()] = 'AR'
df2 = df.loc[df.nacionalidad.str.contains('/') == False].reset_index(drop = True)

In [7]:
df.shape[0] - df2.shape[0] # Se removieron dos sujetos

2

## Conectar con la base de datos y preparar nuestra tabla de consulta

In [8]:
engine = sqlalchemy.create_engine('sqlite:///SQL/CPA.db')
con = engine.connect()

In [9]:
cpa_prov = pd.read_sql('provincia', con=con)

In [10]:
cpa_local = pd.read_sql('localidad', con=con)

C:\Users\joaqu\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1719: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['10']; using no arguments.
  % (coltype, args)
C:\Users\joaqu\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1719: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.SMALLINT'> with reflected arguments ['6']; using no arguments.
  % (coltype, args)


In [11]:
cpa_local[cpa_local.codigopostal == 0] #Se debe remover este campo del dataframe antes de realizar el join. Solo deberia estar en la tabla Provincias

,l_id,provincia_id,nombre,codigopostal
2754,2755,1,C.A.B.A.,0


In [12]:
cpa_local.drop(cpa_local[cpa_local.codigopostal == 0].index).reset_index(drop = True, inplace=True)

In [13]:
cpa_capital =  pd.read_sql('comunas_capital', con=con) # Tabla con localidades de capital

C:\Users\joaqu\Anaconda3\lib\site-packages\sqlalchemy\dialects\sqlite\base.py:1719: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['2']; using no arguments.
  % (coltype, args)


In [14]:
cpa_provincia_completo = cpa_local.merge(cpa_prov, left_on='provincia_id', right_on='p_id', suffixes=['_localidad','_provincia']).drop(columns = 'p_id')
CPA_ARGENTINA =  cpa_provincia_completo.merge(cpa_capital, on= 'provincia_id' ,how='left' , suffixes = ('_x',''),)  # El dataframe mas completo hasta el momento de CPA y localidad

In [15]:
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.fillna(CPA_ARGENTINA.codigopostal_x) # Unir los dos codigos postales de las distintas tablas
CPA_ARGENTINA.drop(columns='codigopostal_x', inplace= True) # eliminar la columna CP repetida
CPA_ARGENTINA['codigopostal'] = CPA_ARGENTINA.codigopostal.astype(int) # Transformar a integer el CP

In [16]:
set_provincia = set(cpa_provincia_completo.codigopostal)
set_provincia.remove(0) # Nuevamente removemos el 0 del codigo postal de provincia
set_capital = set(cpa_capital.codigopostal)
set_capital.intersection(set_provincia) # No hay CP en capital que se repitan en las otras provincias

set()

## Asignar localidad, departamento(comuna CP) y provincia usando CP

In [17]:
# Listado de CP que no son unicos
provincia_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_provincia'].nunique()
cp_prov_no_unicos = set(provincia_por_cp.loc[provincia_por_cp > 1].index)

localidad_por_cp = cpa_provincia_completo.groupby('codigopostal')['nombre_localidad'].nunique()
cp_localidad_no_unicos = set(localidad_por_cp.loc[localidad_por_cp > 1].index) 

capital_barrio_por_cp = cpa_capital.groupby('codigopostal')['barrio'].nunique()
capital_cp_barrio_no_unicos = set(capital_barrio_por_cp.loc[capital_barrio_por_cp > 1].index)

capital_comuna_por_cp = cpa_capital.groupby('codigopostal')['comuna'].nunique()
capital_cp_comuna_no_unicos = set(capital_comuna_por_cp.loc[capital_comuna_por_cp > 1].index)

# Funciones utiles
def infer_provincia(cp):
    if int(cp) in set_capital:
        return 'CIUDAD AUTONOMA DE BUENOS AIRES'
    if int(cp) in set_provincia:
        if  int(cp) in cp_prov_no_unicos:
            return  'NO UNICA PROVINCIA'
        else:
            prov = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == int(cp)].nombre_provincia.unique()
            return prov[0].upper()
    if (int(cp) not in set_provincia and int(cp) not in set_capital):
        return 'CP INDEFINIDO'
    if cp.isna() or int(cp) == 0:
        return 'CP INDEFINIDO'

def infer_localidad(cp):
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
                return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_capital.loc[cpa_capital.codigopostal == cp].barrio.unique()[0]
            return localidad.upper()
    if cp in set_provincia:
        if cp in cp_localidad_no_unicos:
            return 'NO UNICA LOCALIDAD'
        else:
            localidad = cpa_provincia_completo.loc[cpa_provincia_completo.codigopostal == cp].nombre_localidad.unique()[0]
            return localidad.upper()
    if cp not in set_provincia and cp not in set_capital:
        return 'CP INDEFINIDO'
    
def infer_departamento(cp):
    cp = int(cp)
    if cp in set_capital:
        if cp in capital_cp_barrio_no_unicos:
            return 'NO UNICO DEPARTAMENTO'
        else:
            departamento = cpa_capital.loc[cpa_capital.codigopostal == cp].comuna.unique()
            return f'COMUNA {departamento[0]}'

In [119]:
augmented_df = df2.copy() # este es el dataframe que vamos a popular
#Agregamos columnas
augmented_df['provincia_por_cp']= None
augmented_df['localidad_por_cp']= None
augmented_df['departamento_por_cp']= None
augmented_df['provincia_por_API'] = None
augmented_df['localidad_por_API'] = None
augmented_df['departamento_por_API'] = None
augmented_df['centroide_lat_localidad'] = None
augmented_df['centroide_lon_localidad'] = None
augmented_df['centroide_lat_departamento'] = None
augmented_df['centroide_lon_departamento'] = None
augmented_df['centroide_lat_provincia'] = None
augmented_df['centroide_lon_provincia'] = None
augmented_df.loc_part = augmented_df.loc_part.apply(lambda x: x.upper() if isinstance(x,str) else None) #Poner todo en mayuscula

In [120]:
# Provincia
augmented_df['provincia_por_cp'] = augmented_df.cp_part.apply(infer_provincia)
# Localidad
augmented_df['localidad_por_cp'] = augmented_df.cp_part.apply(infer_localidad)
# Departamento 'Comuna'
augmented_df['departamento_por_cp'] = augmented_df.cp_part.apply(infer_departamento)

In [121]:
augmented_df.head()

,nacionalidad,provincia_part,cp_part,loc_part,provincia_por_cp,localidad_por_cp,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,None,None,None,None,None,None,None,None,None
1,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,None,None,None,None,None,None,None,None,None
2,AR,NaN,1688,12 DE AGOSTO,BUENOS AIRES,VILLA SANTOS TESEI,None,None,None,None,None,None,None,None,None,None
3,AR,NaN,6660,25 DE MAYO,BUENOS AIRES,NO UNICA LOCALIDAD,None,None,None,None,None,None,None,None,None,None
4,AR,NaN,0,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,None,None,None,None,None,None,None,None,None,None


In [122]:
#Extraigo los datos de la API 
departamentos = pd.read_csv('https://apis.datos.gob.ar/georef/api/departamentos.csv')
localidades = pd.read_csv('https://apis.datos.gob.ar/georef/api/localidades.csv')
provincias = pd.read_csv('https://apis.datos.gob.ar/georef/api/provincias.csv')

### Conseguir datos de los que tienen provincia y localidad provista por codigo postal

In [65]:
#Funcion que nos va a permitir recorrer el diccionario devuelto
from functools import reduce  
import operator

def getFromDict(dataDict, *args):
    '''
    Esta funcion toma un diccionario y busca la key correspondiente a una concatenacion de keys
    ex: dict[key 1][key 2]...[key n]
    '''
    if len(dataDict) == 0:
        return 'NO HUBO REQUEST'
    else:
        mapList = args
        return reduce(operator.getitem, mapList, dataDict)

In [123]:
# Seleccionamos solo las prov con loc
gby_prov_loc = augmented_df.groupby(['provincia_por_cp','localidad_por_cp']).count().reset_index(level=['provincia_por_cp', 'localidad_por_cp']).copy()
gby_prov_loc = gby_prov_loc.loc[(gby_prov_loc.localidad_por_cp != 'NO UNICA LOCALIDAD') & (gby_prov_loc.localidad_por_cp != 'CP INDEFINIDO')]

In [124]:
gby_prov_loc.drop(columns = ['nacionalidad','provincia_part','cp_part','loc_part','departamento_por_cp'], inplace= True)
gby_prov_loc.loc[:,'provincia_por_API' : 'centroide_lon_provincia'] = None # inicializo todos los campos a zero
print(gby_prov_loc.shape)
gby_prov_loc.head(4)

(83, 11)


,provincia_por_cp,localidad_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia
0,BUENOS AIRES,AEROPUERTO EZEIZA,None,None,None,None,None,None,None,None,None
1,BUENOS AIRES,ALEJANDRO KORN,None,None,None,None,None,None,None,None,None
2,BUENOS AIRES,BANFIELD,None,None,None,None,None,None,None,None,None
3,BUENOS AIRES,BECCAR,None,None,None,None,None,None,None,None,None


In [125]:
# crea una lista de tuplas para realizar el post
prov_loc = list(gby_prov_loc.apply(lambda x: (x.provincia_por_cp, x.localidad_por_cp), axis=1))
# Manda el POST request
datos = geo.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [126]:
# Rellena la tabla con los datos
gby_prov_loc['provincia_por_API'] = [getFromDict(i,'provincia','nombre').upper() for i in datos]
gby_prov_loc['provincia_id_API'] = [getFromDict(i,'provincia','id') for i in datos]
gby_prov_loc['departamento_por_API'] = [getFromDict(i,'departamento','nombre').upper() for i in datos]
gby_prov_loc['departamento_id_API'] = [getFromDict(i,'departamento','id') for i in datos]
gby_prov_loc['localidad_por_API'] = [getFromDict(i,'nombre').upper() for i in datos]
gby_prov_loc['centroide_lat_localidad'] = [getFromDict(i,'centroide','lat') for i in datos]
gby_prov_loc['centroide_lon_localidad'] = [getFromDict(i,'centroide','lon') for i in datos]

In [127]:
gby_prov_loc.head(5)

,provincia_por_cp,localidad_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
0,BUENOS AIRES,AEROPUERTO EZEIZA,BUENOS AIRES,AEROPUERTO INTERNACIONAL EZEIZA,JOSÉ M. EZEIZA,-34.8128,-58.5522,None,None,None,None,06,06270
1,BUENOS AIRES,ALEJANDRO KORN,BUENOS AIRES,ALEJANDRO KORN,SAN VICENTE,-35.0135,-58.3865,None,None,None,None,06,06778
2,BUENOS AIRES,BANFIELD,BUENOS AIRES,BANFIELD,LOMAS DE ZAMORA,-34.7379,-58.4407,None,None,None,None,06,06490
3,BUENOS AIRES,BECCAR,BUENOS AIRES,BECCAR,SAN ISIDRO,-34.4674,-58.54,None,None,None,None,06,06756
4,BUENOS AIRES,BELLA VISTA,BUENOS AIRES,BELLA VISTA,SAN MIGUEL,-34.5751,-58.696,None,None,None,None,06,06760


In [128]:
#Asignar centroide de departamentos
gby_prov_loc.departamento_id_API = gby_prov_loc.departamento_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc['copy_index'] = gby_prov_loc.index
gby_prov_loc.centroide_lat_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_departamento = gby_prov_loc.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

In [129]:
#Asignar centroide de provincias
gby_prov_loc.provincia_id_API = gby_prov_loc.provincia_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_prov_loc.centroide_lat_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lat']
gby_prov_loc.centroide_lon_provincia = gby_prov_loc.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_prov_loc['copy_index'])['centroide_lon']

In [130]:
gby_prov_loc.head(5)

,provincia_por_cp,localidad_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API,copy_index
0,BUENOS AIRES,AEROPUERTO EZEIZA,BUENOS AIRES,AEROPUERTO INTERNACIONAL EZEIZA,JOSÉ M. EZEIZA,-34.8128,-58.5522,-34.875821,-58.564840,-36.676942,-60.558832,6,6270,0
1,BUENOS AIRES,ALEJANDRO KORN,BUENOS AIRES,ALEJANDRO KORN,SAN VICENTE,-35.0135,-58.3865,-35.070460,-58.431922,-36.676942,-60.558832,6,6778,1
2,BUENOS AIRES,BANFIELD,BUENOS AIRES,BANFIELD,LOMAS DE ZAMORA,-34.7379,-58.4407,-34.755471,-58.424079,-36.676942,-60.558832,6,6490,2
3,BUENOS AIRES,BECCAR,BUENOS AIRES,BECCAR,SAN ISIDRO,-34.4674,-58.54,-34.486950,-58.537265,-36.676942,-60.558832,6,6756,3
4,BUENOS AIRES,BELLA VISTA,BUENOS AIRES,BELLA VISTA,SAN MIGUEL,-34.5751,-58.696,-34.552092,-58.691706,-36.676942,-60.558832,6,6760,4


#### Agregar los datos a la tabla aumentada

In [131]:
pd.set_option('display.max_columns', 800)  # Para ver todas las filas
# Vamos a hacer un Join con los valores que tengan la misma provincia y localidad por CP
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales
gby_prov_loc.drop(columns= 'copy_index', inplace= True)

augmented_df = augmented_df.merge(gby_prov_loc , on = ['provincia_por_cp','localidad_por_cp'],
                                  how= 'left', suffixes= ['_del','']).set_index(augmented_df['copy_index']).copy()
                                                                                 
augmented_df = augmented_df.drop(columns = augmented_df.filter(regex = '._del').columns)
augmented_df.drop(columns = ['copy_index'], inplace = True)

In [132]:
augmented_df.head(7)

,nacionalidad,provincia_part,cp_part,loc_part,provincia_por_cp,localidad_por_cp,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
copy_index,,,,,,,,,,,,,,,,,,
0,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,BUENOS AIRES,RAMOS MEJIA,LA MATANZA,-34.6523,-58.5592,-34.770165,-58.625449,-36.676942,-60.558832,6,6427
1,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,BUENOS AIRES,RAMOS MEJIA,LA MATANZA,-34.6523,-58.5592,-34.770165,-58.625449,-36.676942,-60.558832,6,6427
2,AR,NaN,1688,12 DE AGOSTO,BUENOS AIRES,VILLA SANTOS TESEI,None,BUENOS AIRES,VILLA SANTOS TESEI,HURLINGHAM,-34.6124,-58.6538,-34.599214,-58.649694,-36.676942,-60.558832,6,6408
3,AR,NaN,6660,25 DE MAYO,BUENOS AIRES,NO UNICA LOCALIDAD,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AR,NaN,0,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AR,NaN,1839,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AR,NaN,1770,ALDO BONZI,BUENOS AIRES,NO UNICA LOCALIDAD,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Conseguir datos de los que tienen solo provincia por codigo postal

In [33]:
gby_solo_provincia = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida pero si me dio provincia
gby_solo_provincia = gby_solo_provincia.loc[(gby_solo_provincia.provincia_por_cp != 'CP INDEFINIDO') &
                                            (gby_solo_provincia.provincia_por_cp != 'NO UNICA PROVINCIA') &
                                            (gby_solo_provincia.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_provincia.shape)
gby_solo_provincia.head(5)

(445, 18)


,loc_part,provincia_por_cp,localidad_por_cp,nacionalidad,provincia_part,cp_part,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
0,VENEZUELA Y PERU,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
4,-1,SAN JUAN,NO UNICA LOCALIDAD,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
5,-1,SAN LUIS,NO UNICA LOCALIDAD,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0
7,12 DE AGOSTO,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_provincia.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  # no necesito saber que no se sabe la localidad
# En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request
datos = geo.post_centroides('localidades', prov_loc, prov = True) # Consulta get_centroides.py

In [35]:
datos[6:8]

[{'categoria': 'Localidad simple',
  'centroide': {'lat': -37.2020524773603, 'lon': -60.1652139181813},
  'departamento': {'id': '06049', 'nombre': 'Azul'},
  'id': '06049050000',
  'localidad_censal': {'id': '06049050', 'nombre': '16 de Julio'},
  'municipio': {'id': '060049', 'nombre': 'Azul'},
  'nombre': '16 DE JULIO',
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'}},
 {}]

In [36]:
# Rellena la tabla con los datos
gby_solo_provincia['provincia_por_API'] = [getFromDict(i,'provincia','nombre').upper() for i in datos]
gby_solo_provincia['provincia_id_API'] = [getFromDict(i,'provincia','id') for i in datos]
gby_solo_provincia['departamento_por_API'] = [getFromDict(i,'departamento','nombre') for i in datos]
# Algunas queries no devuelven departamentos debido a no existen. Se pasara a CAPS luego
gby_solo_provincia['departamento_id_API'] = [getFromDict(i,'departamento','id') for i in datos]
gby_solo_provincia['localidad_por_API'] = [getFromDict(i,'nombre').upper() for i in datos]
gby_solo_provincia['centroide_lat_localidad'] = [getFromDict(i,'centroide','lat') for i in datos]
gby_solo_provincia['centroide_lon_localidad'] = [getFromDict(i,'centroide','lon') for i in datos]

# Departamento a upper
gby_solo_provincia.departamento_por_API = gby_solo_provincia.departamento_por_API.apply(lambda x: x.upper() if isinstance(x,str) else 'NO HUBO REQUEST')

In [37]:
#Asignar centroide de departamentos
gby_solo_provincia.departamento_id_API = gby_solo_provincia.departamento_id_API.apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and (pd.notna(x)) else x)) # Pasar a integer la olumna
gby_solo_provincia[6:9]

,loc_part,provincia_por_cp,localidad_por_cp,nacionalidad,provincia_part,cp_part,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
12,16 DE JULIO,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,BUENOS AIRES,16 DE JULIO,AZUL,-37.2021,-60.1652,0,0,0,0,06,6049
13,16 DE JULIO,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,0,0,0,0,NO HUBO REQUEST,NO HUBO REQUEST
14,17 DE AGOSTO,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,BUENOS AIRES,17 DE AGOSTO,PUÁN,-37.9087,-62.936,0,0,0,0,06,6651


In [38]:
#Asignar centroide de departamento
gby_solo_provincia['copy_index_2'] = gby_solo_provincia.index
gby_solo_provincia.centroide_lat_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_departamento = gby_solo_provincia.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']

In [39]:
#Asignar centroide de provincias
gby_solo_provincia.provincia_id_API = gby_solo_provincia.provincia_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_solo_provincia.centroide_lat_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lat']
gby_solo_provincia.centroide_lon_provincia = gby_solo_provincia.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_provincia['copy_index_2'])['centroide_lon']

In [40]:
gby_solo_provincia[5:9]

,loc_part,provincia_por_cp,localidad_por_cp,nacionalidad,provincia_part,cp_part,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API,copy_index_2
11,12 DE OCTUBRE,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST,11
12,16 DE JULIO,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,BUENOS AIRES,16 DE JULIO,AZUL,-37.2021,-60.1652,-36.785772,-59.696652,-36.676942,-60.558832,6,6049,12
13,16 DE JULIO,CIUDAD AUTONOMA DE BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,1,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST,13
14,17 DE AGOSTO,BUENOS AIRES,NO UNICA LOCALIDAD,1,1,1,0,BUENOS AIRES,17 DE AGOSTO,PUÁN,-37.9087,-62.936,-38.074604,-63.057746,-36.676942,-60.558832,6,6651,14


In [41]:
departamentos[departamentos.nombre == 'Azul'] #Los centroides para departamento coinciden.

,categoria,centroide_lat,centroide_lon,fuente,id,nombre,nombre_completo,provincia_id,provincia_interseccion,provincia_nombre
150,Partido,-36.785772,-59.696652,ARBA - Gerencia de Servicios Catastrales,6049,Azul,Partido de Azul,6,0.02138,Buenos Aires


###  Conseguir datos de los que no obtuvimos ni provincia ni localidad


In [42]:
gby_solo_loc_part = augmented_df.groupby(['loc_part', 'provincia_por_cp','localidad_por_cp']).count(
                        ).reset_index(['loc_part','provincia_por_cp','localidad_por_cp']) # con este subset armar un post request
# Selecciono las opciones en las cuales el CP no dio una localidad definida, ni provincia o tambien si el CP es indefinido
gby_solo_loc_part = gby_solo_loc_part.loc[(gby_solo_loc_part.provincia_por_cp == 'CP INDEFINIDO') |
                                            (gby_solo_loc_part.provincia_por_cp == 'NO UNICA PROVINCIA') &
                                            (gby_solo_loc_part.localidad_por_cp == 'NO UNICA LOCALIDAD')]
print(gby_solo_loc_part.shape)
gby_solo_loc_part.head(5)

(145, 18)


,loc_part,provincia_por_cp,localidad_por_cp,nacionalidad,provincia_part,cp_part,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
9,12 DE AGOSTO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
21,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0
25,ABASTO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
26,ACASSUSO,CP INDEFINIDO,CP INDEFINIDO,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0
40,ALICIA,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
# Crea una lista de tuplas para realizar el post
prov_loc = list(gby_solo_loc_part.apply(lambda x: (x.provincia_por_cp, x.loc_part), axis=1))  # no necesito saber que no se sabe la localidad
# En vez de localidad usamos loc_part (localidad reportada)
# Manda el POST request

In [44]:
datos = geo.post_centroides('localidades', prov_loc, prov = False) # Consulta get_centroides.py, no uso la provincia en este caso

In [45]:
datos[:2]

[{},
 {'categoria': 'Entidad',
  'centroide': {'lat': -34.7565075768115, 'lon': -58.4902923509723},
  'departamento': {'id': '06260', 'nombre': 'Esteban Echeverría'},
  'id': '06260010005',
  'localidad_censal': {'id': '06260010', 'nombre': 'Esteban Echeverría'},
  'municipio': {'id': '060260', 'nombre': 'Esteban Echeverría'},
  'nombre': '9 DE ABRIL',
  'provincia': {'id': '06', 'nombre': 'Buenos Aires'}}]

In [46]:
# Rellena la tabla con los datos
gby_solo_loc_part['provincia_por_API'] = [getFromDict(i,'provincia','nombre').upper() for i in datos]
gby_solo_loc_part['provincia_id_API'] = [getFromDict(i,'provincia','id') for i in datos]
gby_solo_loc_part['departamento_por_API'] = [getFromDict(i,'departamento','nombre') for i in datos]
# Algunas queries no devuelven departamentos debido a no existen. Se pasara a CAPS luego
gby_solo_loc_part['departamento_id_API'] = [getFromDict(i,'departamento','id') for i in datos]
gby_solo_loc_part['localidad_por_API'] = [getFromDict(i,'nombre').upper() for i in datos]
gby_solo_loc_part['centroide_lat_localidad'] = [getFromDict(i,'centroide','lat') for i in datos]
gby_solo_loc_part['centroide_lon_localidad'] = [getFromDict(i,'centroide','lon') for i in datos]

# Departamento a upper
gby_solo_loc_part.departamento_por_API = gby_solo_loc_part.departamento_por_API.apply(lambda x: x.upper() if isinstance(x,str) else 'NO HUBO REQUEST')

In [47]:
#Asignar centroide de departamentos
gby_solo_loc_part.departamento_id_API = gby_solo_loc_part.departamento_id_API.apply(lambda x: (int(x) if  (x != 'NO HUBO REQUEST') and (pd.notna(x)) else x)) # Pasar a integer la columna
gby_solo_loc_part['copy_index_2'] = gby_solo_loc_part.index
gby_solo_loc_part.centroide_lat_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_departamento = gby_solo_loc_part.merge(departamentos, left_on= 'departamento_id_API', right_on='id', how ='left').set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']

In [48]:
#Asignar centroide de provincias
gby_solo_loc_part.provincia_id_API = gby_solo_loc_part.provincia_id_API.apply(lambda x: (int(x) if  x != 'NO HUBO REQUEST' else x)) # Pasar a integer la columna
gby_solo_loc_part.centroide_lat_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lat']
gby_solo_loc_part.centroide_lon_provincia = gby_solo_loc_part.merge(provincias, left_on= 'provincia_id_API', right_on='id', how='left' ).set_index(gby_solo_loc_part['copy_index_2'])['centroide_lon']

In [49]:
gby_solo_loc_part[gby_solo_loc_part['localidad_por_API'] == 'NO HUBO REQUEST'].shape #27 request no devolvieron resultados

(27, 19)

In [50]:
gby_solo_loc_part.head()

,loc_part,provincia_por_cp,localidad_por_cp,nacionalidad,provincia_part,cp_part,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API,copy_index_2
9,12 DE AGOSTO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST,9
21,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,2,0,2,0,BUENOS AIRES,9 DE ABRIL,ESTEBAN ECHEVERRÍA,-34.7565,-58.4903,-34.831210,-58.476969,-36.676942,-60.558832,6,6260,21
25,ABASTO,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,BUENOS AIRES,ABASTO,LA PLATA,-35.0104,-58.1152,-35.003494,-58.018027,-36.676942,-60.558832,6,6441,25
26,ACASSUSO,CP INDEFINIDO,CP INDEFINIDO,2,2,2,0,BUENOS AIRES,ACASSUSO,SAN ISIDRO,-34.4794,-58.5074,-34.486950,-58.537265,-36.676942,-60.558832,6,6756,26
40,ALICIA,CP INDEFINIDO,CP INDEFINIDO,1,1,1,0,CÓRDOBA,ALICIA,SAN JUSTO,-31.9387,-62.4658,-31.239399,-62.526029,-32.142933,-63.801753,14,14140,40


## Agregar todos los datos a la tabla

In [143]:
aux_df = gby_solo_loc_part.append(gby_solo_provincia, sort = False) #append all tables in one df. Except prov_loc 

In [144]:
gby_solo_loc_part.shape[0] + gby_solo_provincia.shape[0] == aux_df.shape[0]

True

In [149]:
# Vamos a hacer un mege con los valores que tengan la misma provincia y localidad por CP y lat_long
augmented_df['copy_index'] = augmented_df.index.copy()  # Guardar los index iniciales

augmented_df2 = augmented_df.merge(aux_df , on = ['loc_part','provincia_por_cp','localidad_por_cp'],
                                  how= 'left', suffixes= ['','_added']).set_index(augmented_df['copy_index']).copy()


In [150]:
augmented_df2.head()

,nacionalidad,provincia_part,cp_part,loc_part,provincia_por_cp,localidad_por_cp,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API,copy_index,nacionalidad_added,provincia_part_added,cp_part_added,departamento_por_cp_added,provincia_por_API_added,localidad_por_API_added,departamento_por_API_added,centroide_lat_localidad_added,centroide_lon_localidad_added,centroide_lat_departamento_added,centroide_lon_departamento_added,centroide_lat_provincia_added,centroide_lon_provincia_added,provincia_id_API_added,departamento_id_API_added,copy_index_2
copy_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,BUENOS AIRES,RAMOS MEJIA,LA MATANZA,-34.6523,-58.5592,-34.770165,-58.625449,-36.676942,-60.558832,6,6427,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AR,NaN,1704,None,BUENOS AIRES,RAMOS MEJIA,None,BUENOS AIRES,RAMOS MEJIA,LA MATANZA,-34.6523,-58.5592,-34.770165,-58.625449,-36.676942,-60.558832,6,6427,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,NaN,1688,12 DE AGOSTO,BUENOS AIRES,VILLA SANTOS TESEI,None,BUENOS AIRES,VILLA SANTOS TESEI,HURLINGHAM,-34.6124,-58.6538,-34.599214,-58.649694,-36.676942,-60.558832,6,6408,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AR,NaN,6660,25 DE MAYO,BUENOS AIRES,NO UNICA LOCALIDAD,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3.0,2.0,3.0,0.0,BUENOS AIRES,25 DE MAYO,25 DE MAYO,-35.4339,-60.1731,-35.526511,-60.230158,-36.676942,-60.558832,6,6854,17.0
4,AR,NaN,0,9 DE ABRIL,CP INDEFINIDO,CP INDEFINIDO,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2.0,0.0,2.0,0.0,BUENOS AIRES,9 DE ABRIL,ESTEBAN ECHEVERRÍA,-34.7565,-58.4903,-34.831210,-58.476969,-36.676942,-60.558832,6,6260,21.0


In [151]:
def assign_value(row, column = None):
    if pd.isna(row[column]):
        return row[column + '_added']
    else:
        return row[column]

In [152]:
augmented_df2['provincia_por_API'] = augmented_df2.apply(assign_value, column = 'provincia_por_API', axis = 1)
augmented_df2['localidad_por_API'] =  augmented_df2.apply(assign_value, column = 'localidad_por_API', axis = 1)
augmented_df2['departamento_por_API'] = augmented_df2.apply(assign_value, column = 'departamento_por_API', axis = 1)
augmented_df2['centroide_lat_localidad'] = augmented_df2.apply(assign_value, column = 'centroide_lat_localidad', axis = 1)
augmented_df2['centroide_lon_localidad'] = augmented_df2.apply(assign_value, column = 'centroide_lon_localidad', axis = 1)
augmented_df2['centroide_lat_departamento'] = augmented_df2.apply(assign_value, column = 'centroide_lat_departamento', axis = 1)
augmented_df2['centroide_lon_departamento'] = augmented_df2.apply(assign_value, column = 'centroide_lon_departamento', axis = 1)
augmented_df2['centroide_lat_provincia'] = augmented_df2.apply(assign_value, column = 'centroide_lat_provincia', axis = 1)
augmented_df2['centroide_lon_provincia'] = augmented_df2.apply(assign_value, column = 'centroide_lon_provincia', axis = 1)
augmented_df2['provincia_id_API'] = augmented_df2.apply(assign_value, column = 'provincia_id_API', axis = 1)
augmented_df2['departamento_id_API'] = augmented_df2.apply(assign_value, column = 'departamento_id_API', axis = 1)

In [167]:
augmented_df2.drop(columns = augmented_df2.filter(regex = '._added').columns, inplace=True)
augmented_df2.drop(columns = ['copy_index', 'copy_index_2'], inplace = True)

In [172]:
augmented_df2.sample(5)

,nacionalidad,provincia_part,cp_part,loc_part,provincia_por_cp,localidad_por_cp,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
copy_index,,,,,,,,,,,,,,,,,,
657,AR,BA,1602,LA FLORIDA,BUENOS AIRES,NO UNICA LOCALIDAD,None,BUENOS AIRES,FLORIDA,VICENTE LÓPEZ,-34.5327,-58.4911,-34.526513,-58.504470,-36.676942,-60.558832,6,6861
737,AR,BA,1768,MACEDO,BUENOS AIRES,CIUDAD MADERO,None,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST
119,AR,NaN,0,INGENIERO ADOLFO SOU,CP INDEFINIDO,CP INDEFINIDO,None,BUENOS AIRES,INGENIERO ADOLFO SOURDEAUX,MALVINAS ARGENTINAS,-34.5004,-58.6618,-34.487304,-58.712133,-36.676942,-60.558832,6,6515
786,AR,BA,1744,MORENO,BUENOS AIRES,NO UNICA LOCALIDAD,None,BUENOS AIRES,MORENO,MORENO,-34.6376,-58.7921,-34.610617,-58.810923,-36.676942,-60.558832,6,6560
45,AR,NaN,1605,CARAPACHAY,BUENOS AIRES,NO UNICA LOCALIDAD,None,BUENOS AIRES,CARAPACHAY,VICENTE LÓPEZ,-34.5259,-58.5358,-34.526513,-58.504470,-36.676942,-60.558832,6,6861


In [171]:
departamentos[departamentos.nombre == 'Pergamino']

,categoria,centroide_lat,centroide_lon,fuente,id,nombre,nombre_completo,provincia_id,provincia_interseccion,provincia_nombre
257,Partido,-33.835816,-60.545075,ARBA - Gerencia de Servicios Catastrales,6623,Pergamino,Partido de Pergamino,6,0.009484,Buenos Aires


## EDA de la tabla final

In [156]:
augmented_df2[augmented_df2['localidad_por_API'] == 'NO HUBO REQUEST'].shape

(180, 35)

In [157]:
augmented_df2[augmented_df2['provincia_por_API'] == 'NO HUBO REQUEST'].shape

(180, 35)

In [155]:
augmented_df2.shape

(1425, 35)

Aproximadamente un 12% no pudo ser identificado. Debemos remarcar el hecho de que usamos solamente 'casos unicos' para la combinacion `cp_part`, `loc_part` y `provincia_part` de la base original del HI. Es esperable que en la practica este numero sea menor. 
Tambien tenemos que mencionar que confiar en el codigo postal lejos esta de ser el mejor de los casos (los sujetos pueden olvidarlo o ingresarlo erroneamente sin darse cuenta) pero frente los inconvenientes debidos a la falta de normalizacion de la base inicial nos parecio la mejor opcion. 

In [175]:
augmented_df2.sample(10)

,nacionalidad,provincia_part,cp_part,loc_part,provincia_por_cp,localidad_por_cp,departamento_por_cp,provincia_por_API,localidad_por_API,departamento_por_API,centroide_lat_localidad,centroide_lon_localidad,centroide_lat_departamento,centroide_lon_departamento,centroide_lat_provincia,centroide_lon_provincia,provincia_id_API,departamento_id_API
copy_index,,,,,,,,,,,,,,,,,,
726,AR,BA,2814,LOS CARDALES,BUENOS AIRES,NO UNICA LOCALIDAD,None,BUENOS AIRES,LOS CARDALES,EXALTACIÓN DE LA CRUZ,-34.3301,-58.9886,-34.295149,-59.155912,-36.676942,-60.558832,6,6266
267,AR,NaN,1611,TIGRE,BUENOS AIRES,DON TORCUATO,None,BUENOS AIRES,DON TORCUATO ESTE,TIGRE,-34.4869,-58.6159,-34.381589,-58.599454,-36.676942,-60.558832,6,6805
804,AR,BA,7400,OLAVARRIA,BUENOS AIRES,NO UNICA LOCALIDAD,None,BUENOS AIRES,OLAVARRIA,OLAVARRÍA,-36.8921,-60.318,-36.856701,-60.670735,-36.676942,-60.558832,6,6595
647,AR,BA,1655,JOSE LEON SUAREZ,BUENOS AIRES,JOSE LEON SUAREZ,None,BUENOS AIRES,VILLA JOSE LEON SUAREZ,CIUDAD LIBERTADOR SAN MARTÍN,-34.5254,-58.5803,-34.552649,-58.564315,-36.676942,-60.558832,6,6371
1367,AR,SA,4000,EL MIRADOR,TUCUMÁN,NO UNICA LOCALIDAD,None,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST
111,AR,NaN,1706,GUIDO SPANO,BUENOS AIRES,NO UNICA LOCALIDAD,None,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NO HUBO REQUEST,NaN,NaN,NaN,NaN,NO HUBO REQUEST,NO HUBO REQUEST
897,AR,BA,1718,SAN A DE PADUA,BUENOS AIRES,SAN ANTONIO DE PADUA,None,BUENOS AIRES,SAN ANTONIO DE PADUA,MERLO,-34.6699,-58.6979,-34.710887,-58.741878,-36.676942,-60.558832,6,6539
199,AR,NaN,0,OLIVOS,CP INDEFINIDO,CP INDEFINIDO,None,BUENOS AIRES,OLIVOS,VICENTE LÓPEZ,-34.5122,-58.4985,-34.526513,-58.504470,-36.676942,-60.558832,6,6861
1320,AR,LP,6205,INGENIERO LUIGGI,LA PAMPA,NO UNICA LOCALIDAD,None,LA PAMPA,INGENIERO LUIGGI,REALICÓ,-35.3838,-64.4686,-35.225792,-64.210227,-37.131554,-65.446655,42,42133
